In [ ]:
#@title Codes for JAFTA internally. { display-mode: "form" }


<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/thanhgis/PeatLands/blob/main/runme.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/thanhgis/PeatLands/main/runme.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

### Step 1: Import the API
You can run this step one time only if you are working on the same session. 

Please click on the hyperlink in below codeshell, then follow-up the guidance in the newly-opened tab to authenticate, then copy (click on 'Copy' button) the code and come back to this part to paste into the blank area and press 'Enter' keyboard to continue.

In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

### Step 2: Import the algorithms and library

Please run below step in order to import the algorithms and liberary into this Colab working space. If you do not change the codes in GitHub, just run this one time. However, if you made some changes in GitHub, please reload Runtime (Click 'Runtime' menu above, then select 'Restart runtime') then run this step again.

In [13]:
import requests 
derive = 'https://raw.githubusercontent.com/thanhgis/PeatLands/main/pysmm/derive_SM.py'
wrapper = 'https://raw.githubusercontent.com/thanhgis/PeatLands/main/pysmm/GEE_wrappers.py'
decisiontree = 'https://raw.githubusercontent.com/thanhgis/PeatLands/main/pysmm/no_GLDAS_decisiontree_GEE__1step.py'

r1 = requests.get(derive)
r2 = requests.get(wrapper)
r3 = requests.get(decisiontree)

with open ('derive_SM.py', 'w') as f: 
  f.write(r1.text)
with open ('GEE_wrappers.py', 'w') as f: 
  f.write(r2.text)
with open ('no_GLDAS_decisiontree_GEE__1step.py', 'w') as f: 
  f.write(r3.text)

import GEE_wrappers as wrappers
import derive_SM as derive
import no_GLDAS_decisiontree_GEE__1step as decisiontree

%reload_ext autoreload
%autoreload 2

### Step 3: Select area of interest and time of interest.

Please make some changes in this part if you want, then run it.

In [8]:
#Set Area of Interest:  
min_lon = 102.8895
min_lat = -0.1407
max_lon = 103.3895
max_lat = 0.4407

# Set time of Interest:
yyyy = 2020
mm = 3
dd = 1

### Step 4: Run the processing:

Please run this step to process data. This step will also send the outputs to your GEE Asset. If you run the same AOI and date, this step will be ignored.

In [ ]:
from derive_SM import get_map
asset_ID = ee.data.getAssetRoots()[0]['id'].replace('projects/earthengine-legacy/assets/', '')+ '/'
get_map(min_lon, min_lat, max_lon, max_lat,
        asset_ID,
        sampling=30,
        year=yyyy, month=mm, day=dd,
        tracknr=None,
        overwrite=False,
        start=None,
        stop=None)  


### Step 5: Show outputs on the map

Please run this step to show the outputs on a map.

In [ ]:
from GEE_wrappers import GEE_extent
import folium 
roi = ee.Geometry.Polygon([[min_lon, min_lat], [min_lon, max_lat], [max_lon, max_lat], [max_lon, min_lat], [min_lon, min_lat]])
# ThanhGIS added:
centroid =roi.centroid().coordinates()
pointinfo = centroid.getInfo() 
centerlat = pointinfo[0]
centerlon = pointinfo[1]

GEE_interface = GEE_extent(min_lon, min_lat, max_lon, max_lat, workdir=None, sampling=None)
l8_mean = ee.Image(GEE_interface.check_layer()[0])
s1_mean = GEE_interface.check_layer()[1]
evi_mean = GEE_interface.check_layer()[2]
print (l8_mean.get('system:id').getInfo())

#Add EE drawing method to folium.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True).add_to(self)
vis_evi = {'min': -650,'max': 5000}
vis_l8 = {'min': 200, 'max': 4000}
vis_s1 = {'palette': 'red'}
folium.Map.add_ee_layer = add_ee_layer
my_map = folium.Map(location=[centerlon, centerlat], zoom_start=10)
my_map.add_ee_layer(evi_mean, vis_evi, 'EVI_MODIS_Mean')
my_map.add_ee_layer(l8_mean, vis_l8, 'L8_Mean')
my_map.add_ee_layer(s1_mean, vis_s1, 'Detected')
# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
# Display the map.
display(my_map)